To detect fake currency using a Convolutional Neural Network (CNN) in Python, you can follow these steps:

Prepare the dataset:

Collect a dataset of genuine currency images and fake currency images.
Organize the dataset into two separate folders, one for genuine images and one for fake images.
Make sure the images are labeled correctly.
Set up the environment:

Install the required libraries, such as TensorFlow and Keras.
Load the dataset:

Use an image processing library, such as OpenCV, to read and preprocess the images.
Split the dataset into training and testing sets.
Build the CNN model:

Import the necessary modules from TensorFlow and Keras.
Define the architecture of the CNN model, including convolutional layers, pooling layers, and fully connected layers.
Compile the model by specifying the loss function, optimizer, and metrics.
Train the model:

Use the training set to train the CNN model.
Specify the number of epochs and batch size.
Monitor the training process and adjust hyperparameters if needed.
Evaluate the model:

Use the testing set to evaluate the performance of the trained model.
Calculate metrics such as accuracy, precision, and recall.

In [2]:
import numpy as np
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

# Set the path to your dataset
dataset_path = 'noteZip/'

# Define the classes (labels)
classes = ['fake', 'genuine']

# Create empty lists to store the images and labels
images = []
labels = []

# Load the dataset
for cls in classes:
    cls_path = os.path.join(dataset_path, cls)
    for image_name in os.listdir(cls_path):
        image_path = os.path.join(cls_path, image_name)
        image = Image.open(image_path)
        image = image.resize((128, 128))  # Resize the image if needed
        image = np.array(image)
        images.append(image)
        labels.append(classes.index(cls))

# Convert the lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Split the dataset into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Normalize pixel values to be between 0 and 1
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Define the CNN model architecture
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(2, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(test_images, test_labels))

model.save('currency_detections.h5') #pre-trained saved model

Epoch 1/10


C:\Users\chine\anaconda3\lib\site-packages\keras\backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


18/18 [==============================] - 5s 230ms/step - loss: 0.6611 - accuracy: 0.8404 - val_loss: 0.5352 - val_accuracy: 0.7801
Epoch 2/10
18/18 [==============================] - 4s 216ms/step - loss: 0.3710 - accuracy: 0.8528 - val_loss: 0.3622 - val_accuracy: 0.8511
Epoch 3/10
18/18 [==============================] - 4s 217ms/step - loss: 0.2887 - accuracy: 0.8901 - val_loss: 0.2964 - val_accuracy: 0.8369
Epoch 4/10
18/18 [==============================] - 4s 215ms/step - loss: 0.1638 - accuracy: 0.9379 - val_loss: 0.4756 - val_accuracy: 0.8369
Epoch 5/10
18/18 [==============================] - 4s 217ms/step - loss: 0.1363 - accuracy: 0.9468 - val_loss: 0.1423 - val_accuracy: 0.9433
Epoch 6/10
18/18 [==============================] - 4s 218ms/step - loss: 0.0949 - accuracy: 0.9628 - val_loss: 0.0824 - val_accuracy: 0.9645
Epoch 7/10
18/18 [==============================] - 4s 216ms/step - loss: 0.0541 - accuracy: 0.9805 - val_loss: 0.3707 - val_accuracy: 0.8582
Epoch 8/10
18/18 

In [3]:
from flask import Flask, request
from werkzeug.utils import secure_filename
import os, numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from PIL import Image

app = Flask(__name__)

#Load developed machine learning modea and parameter settings

tf.function(
    func=None,
    input_signature=None,
    autograph=True,
    jit_compile=None,
    reduce_retracing=False,
    experimental_implements=None,
    experimental_autograph_options=None,
    experimental_attributes=None,
    experimental_relax_shapes=None,
    experimental_compile=None,
    experimental_follow_type_hints=None
) # tensorflow parameter settings

model = tf.keras.models.load_model('currency_detections.h5') #load pre-trained saved model


@app.route('/upload', methods=['POST'])
def upload():
    if 'image' not in request.files:
        return 'No image file provided', 400

    image = request.files['image']
    if image.filename == '':
        return 'No selected image file', 400

    # Process the image file as required
    filename = secure_filename(image.filename)
    filename, extension = os.path.splitext(filename)
    count = len('upload/') + 1 # count the numbers of files in the directory
    new_name = f'{count}{extension}' # rename image 
    image.save('uploads/'+new_name) # save the file to directory
    
    resp = {}
    
    # Make predictions on new images
    # new_image_path = 'noteZip/200_1.jpg'
    new_image = Image.open('uploads/'+str(count)+extension)
    new_image = new_image.resize((128, 128))
    new_image = np.array(new_image) / 255.0
    new_image = np.expand_dims(new_image, axis=0)

    predictions = model.predict(new_image)
    predicted_label = classes[np.argmax(predictions)]

    print('Predicted label:', predicted_label)
    resp['status']  = predicted_label
    return resp['status']

if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.67.112:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\chine\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
